In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler, Adam

import pytorch_lightning as pl
from pytorch_lightning import Trainer
#from pytorch_lightning.callbacks import ModelCheckpoint
#from pytorch_lightning.loggers import TensorBoardLogger

import pandas as pd
import string


In [3]:
import glob
import os, string
from __future__ import unicode_literals, print_function, division
import unicodedata

from io import open


In [4]:
#get data
def findFiles(path): return glob.glob('C:/Users/fiv24242/Downloads/data/data/names/*.txt')

findFiles('C:/Users/fiv24242/Downloads/data/data/names')

['C:/Users/fiv24242/Downloads/data/data/names\\Arabic.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\Chinese.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\Czech.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\Dutch.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\English.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\French.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\German.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\Greek.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\Irish.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\Italian.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\Japanese.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\Korean.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\Polish.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\Portuguese.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\Russian.txt',
 'C:/Users/fiv24242/Downloads/data/data/names\\Scottish.txt',
 'C:/Users/fiv24242/Downloads/da

In [5]:
all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )



def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]


In [6]:
for filename in findFiles('*.txt'):
    print(filename)

C:/Users/fiv24242/Downloads/data/data/names\Arabic.txt
C:/Users/fiv24242/Downloads/data/data/names\Chinese.txt
C:/Users/fiv24242/Downloads/data/data/names\Czech.txt
C:/Users/fiv24242/Downloads/data/data/names\Dutch.txt
C:/Users/fiv24242/Downloads/data/data/names\English.txt
C:/Users/fiv24242/Downloads/data/data/names\French.txt
C:/Users/fiv24242/Downloads/data/data/names\German.txt
C:/Users/fiv24242/Downloads/data/data/names\Greek.txt
C:/Users/fiv24242/Downloads/data/data/names\Irish.txt
C:/Users/fiv24242/Downloads/data/data/names\Italian.txt
C:/Users/fiv24242/Downloads/data/data/names\Japanese.txt
C:/Users/fiv24242/Downloads/data/data/names\Korean.txt
C:/Users/fiv24242/Downloads/data/data/names\Polish.txt
C:/Users/fiv24242/Downloads/data/data/names\Portuguese.txt
C:/Users/fiv24242/Downloads/data/data/names\Russian.txt
C:/Users/fiv24242/Downloads/data/data/names\Scottish.txt
C:/Users/fiv24242/Downloads/data/data/names\Spanish.txt
C:/Users/fiv24242/Downloads/data/data/names\Vietnamese.t

In [7]:
category_lines = {}
all_categories = []
for filename in findFiles('*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))


# categories: 18 ['Arabic', 'Chinese', 'Czech', 'Dutch', 'English', 'French', 'German', 'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Scottish', 'Spanish', 'Vietnamese']
O'Neal


In [8]:
##RNN network

import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [9]:
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line


In [10]:
# One-hot vector for category
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)


In [11]:
# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor


In [12]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item() / input_line_tensor.size(0)


In [13]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


In [14]:
n_letters

59

In [20]:
rnn = RNN(n_letters, 128, n_letters)

n_iters = 10000
print_every = 5000
plot_every = 50
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0


0m 48s (5000 50%) 3.1307
1m 34s (10000 100%) 3.0093


In [19]:
all_losses

[4.067895335091484,
 4.045254454839798,
 4.02095142313309,
 4.015418033767744,
 3.994731605640407,
 3.969527399844448,
 3.939546195393516,
 3.90472912475729,
 3.899047882532832,
 3.8385706376630333,
 3.826824442395572,
 3.7533343514116293,
 3.730210746572473,
 3.63412600174886,
 3.506137456212725,
 3.4085620949154816,
 3.402116886480203,
 3.40397608981745,
 3.324488096109201,
 3.2853380423170147]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)


In [35]:
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

samples('Russian', 'RUS')

#samples('German', 'GER')

#samples('English', 'ASD')

samples('Chinese', 'CHI')


Ranee
Uanee
Sanee
Can
Han
Ian
